In [ ]:
import face_recognition
import cv2
import numpy as np
import os
from google.colab.patches import cv2_imshow
from PIL import Image

In [ ]:
# Open the input movie file
input_movie = cv2.VideoCapture("test.mp4")
length = int(input_movie.get(cv2.CAP_PROP_FRAME_COUNT))

In [ ]:
# Create an output movie file (make sure resolution/frame rate matches input video!)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_movie = cv2.VideoWriter('output.avi', fourcc, 29.97, (426, 240))

In [ ]:
def get_encoded_faces():
    """
    looks through the faces folder and encodes all
    the faces
 
    :return: dict of (name, image encoded)
    """
    encoded = {}
 
    for dirpath, dnames, fnames in os.walk("./faces"):
        for f in fnames:
            if f.endswith(".jpg") or f.endswith(".png"):
                face = face_recognition.load_image_file("faces/" + f)
                encoding = face_recognition.face_encodings(face)[0]
                encoded[f.split(".")[0]] = encoding
    return encoded

In [ ]:
def unknown_image_encoded(img):
    """
    encode a face given the file name
    """
    face = fr.load_image_file("faces/" + img)
    encoding = fr.face_encodings(face)[0]
 
    return encoding

In [ ]:
# Initialize some variables
face_locations = []
face_encodings =[]
faces = get_encoded_faces()
known_face_encodings = list(faces.values())
known_face_names = list(faces.keys())
frame_number = 0

In [ ]:
while True:
    # Grab a single frame of video
    ret, frame = input_movie.read()
    frame_number += 1

    # Quit when the input video file ends
    if not ret:
        break

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_frame = frame[:, :, ::-1]

    # Find all the faces and face encodings in the current frame of video
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    face_names = []
    for face_encoding in face_encodings:
        # See if the face is a match for the known face(s)
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"

            # # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = known_face_names[best_match_index]
            acc = np.array(known_face_encodings[best_match_index] / face_encoding)*100
            accuracy = round(np.mean(acc),2)
            acc_text = "{}%".format(accuracy)

        face_names.append(name)

    # Label the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        if not name:
            continue

        # Draw a box around the face
        if name == "Unknown":
          cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        else:
          cv2.rectangle(frame, (left, top), (right, bottom), (255, 0, 0), 2)

        # Draw a label with a name below the face
        if name == "Unknown":
          cv2.rectangle(frame, (left, bottom - 25), (right, bottom), (0, 0, 255), cv2.FILLED)
        else:
          cv2.rectangle(frame, (left, bottom - 25), (right, bottom), (255, 0, 0), cv2.FILLED)
        font = cv2.FONT_HERSHEY_COMPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)
        #cv2.putText(frame, acc_text, (right - 6,bottom + 6), font,1.0, (255,255,255),1)

    # Write the resulting image to the output video file
    print("Writing frame {} / {}".format(frame_number, length))
    output_movie.write(frame)

    
    key = cv2.waitKey(1) & 0xFF
	  # if the `q` key was pressed, break from the loop
    if key == ord("q"):
      break

In [ ]:
# do a bit of cleanup
print("[INFO] cleaning up...")
cv2.destroyAllWindows()
output_movie.release()